In [1]:
!pip install mediapipe

  Using cached opencv_contrib_python-4.6.0.66-cp36-abi3-win_amd64.whl (42.5 MB)


In [4]:
!conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0 -y

^C


In [5]:
!pip install "tensorflow<2.11"

In [2]:
!pip install depthai

You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
#open pose
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt
import depthai as dai

In [2]:
def getMonoCamera(pipeline, isLeft):
    #configure mono camera
    mono = pipeline.createMonoCamera()

    #set the Camera Resolution
    mono.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)
    if isLeft:
        mono.setBoardSocket(dai.CameraBoardSocket.LEFT)
    else:
        mono.setBoardSocket(dai.CameraBoardSocket.RIGHT)
    return mono

In [3]:
#define a pipeline
pipeline = dai.Pipeline()

#set up left and right cameras
monoLeft = getMonoCamera(pipeline, isLeft = True)
monoRight = getMonoCamera(pipeline, isLeft = False)

#set output Xlink for left camera
xoutLeft = pipeline.createXLinkOut()
xoutLeft.setStreamName("left")

#set output Xlink for right camera
xoutRight = pipeline.createXLinkOut()
xoutRight.setStreamName("right")

#Attach cameras to output XLink 
monoLeft.out.link(xoutLeft.input)
monoRight.out.link(xoutRight.input)

In [4]:
# create pose detection model
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [5]:
frame_shape = (640,480)
width = frame_shape[0]
height = frame_shape[1]
baseline = 7.5
focal_length = 3.818394999478380e+02 * 20 / 3
i = 1

camera_intrinsics = np.reshape(np.array([3.587843993302980e+02,0,0,0,3.625294262595207e+02,0,2.971900222063954e+02,2.760362029054561e+02,1]), (3,3))
camera_intrinsics


array([[358.78439933,   0.        ,   0.        ],
       [  0.        , 362.52942626,   0.        ],
       [297.19002221, 276.03620291,   1.        ]])

In [6]:

with dai.Device(pipeline) as device:
    leftQueue = device.getOutputQueue(name = 'left', maxSize=4)
    rightQueue = device.getOutputQueue(name = 'right', maxSize = 4)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while True:
            in_left = leftQueue.get()
            in_right = rightQueue.get()

            frame_left = in_left.getCvFrame()
            frame_right = in_right.getCvFrame()
            # improve speed by not rewriting frame
            frame_left = cv2.cvtColor(frame_left, cv2.COLOR_GRAY2RGB)
            frame_right = cv2.cvtColor(frame_right, cv2.COLOR_GRAY2RGB)

            if cv2.waitKey(10) == ord('w'):
                cv2.imwrite('calibration_img_'+str(i)+'.png', frame_left)
                i = i+1

            frame_left.flags.writeable = False
            frame_right.flags.writeable = False

            left_poses = pose.process(frame_left)
            right_poses = pose.process(frame_right)
            
            

            # mp_drawing.draw_landmarks(
            #     frame_left, left_poses.pose_landmarks, 
            #     mp_pose.POSE_CONNECTIONS,
            #     landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            # mp_drawing.draw_landmarks(
            #     frame_right, right_poses.pose_landmarks, 
            #     mp_pose.POSE_CONNECTIONS,
            #     landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            

            #add Left image wrists
            left_img_landmarks = {}
            if left_poses and left_poses.pose_landmarks and left_poses.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST] :
                a = left_poses.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
                left_img_landmarks['left_wrist'] = (int(a.x*width), int(a.y*height))
            else:
                left_img_landmarks['left_wrist'] = None
            if left_poses and left_poses.pose_landmarks and left_poses.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST] :
                a = left_poses.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
                left_img_landmarks['right_wrist'] = (int(a.x*width), int(a.y*height))
            else:
                left_img_landmarks['right_wrist'] = None

            #add Right image wrists
            right_img_landmarks = {}
            if right_poses and right_poses.pose_landmarks and right_poses.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST] :
                a = right_poses.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
                right_img_landmarks['left_wrist'] = (int(a.x*width), int(a.y*height))
            else:
                right_img_landmarks['left_wrist'] = None
            if right_poses and right_poses.pose_landmarks and right_poses.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST] :
                a = right_poses.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
                right_img_landmarks['right_wrist'] = (int(a.x*width), int(a.y*height))
            else:
                right_img_landmarks['right_wrist'] = None

            
            if left_img_landmarks['left_wrist']  and left_img_landmarks['right_wrist']:
                cv2.line(frame_left, left_img_landmarks['left_wrist'], left_img_landmarks['right_wrist'], (0, 255, 0), 2)
            cv2.imshow("left", frame_left)

            if right_img_landmarks['left_wrist']  and right_img_landmarks['right_wrist']:
                cv2.line(frame_right, right_img_landmarks['left_wrist'], right_img_landmarks['right_wrist'], (0, 255, 0), 2)
            cv2.imshow('right', frame_right)

            if left_img_landmarks['left_wrist']  and left_img_landmarks['right_wrist'] and right_img_landmarks['left_wrist']  and right_img_landmarks['right_wrist']:
                left_disparity_x = left_img_landmarks['left_wrist'][0] - left_img_landmarks['right_wrist'][0]
                right_disparity_x = right_img_landmarks['left_wrist'][0] - right_img_landmarks['right_wrist'][0]
                left_disparity_y = left_img_landmarks['left_wrist'][1] - left_img_landmarks['right_wrist'][1]
                right_disparity_y = right_img_landmarks['left_wrist'][1] - right_img_landmarks['right_wrist'][1]
                if left_disparity_x != 0 and right_disparity_x != 0 and left_disparity_y != 0 and right_disparity_y != 0:
                    #calculate real world dimensions from intrinsic matrix and disparity
                    left_real_world_x = (camera_intrinsics[0][0] ) / left_disparity_x
                    right_real_world_x = (camera_intrinsics[0][0] ) / right_disparity_x
                    left_real_world_y = (camera_intrinsics[1][1] ) / left_disparity_y
                    right_real_world_y = (camera_intrinsics[1][1] ) / right_disparity_y

                    #calculate real world distance between wrists
                    real_world_distance = np.sqrt((left_real_world_x - right_real_world_x)**2 + (left_real_world_y - right_real_world_y)**2)
                    print(real_world_distance)
            if cv2.waitKey(1) == ord('q'):
                break
cv2.destroyAllWindows()

34.05723341615989
129.4749356264296
2.3959471285210654
9.716805839155205
84.59142406702557
358.0537576865667
0.46445428617908074
35.450839344730156
87.00735044283455
113.73475744365729
134.27016692192444
36.885478903798976
345.2661443930285
109.51413000278953
12.292406466891158
5.7249974126276255
32.049853020468
217.51789603414898
84.74750215186813
4.030218507005255
37.857140139773556
322.2484349153183
88.04372956194737
15.105617730537862
41.10096154892681
6.664049641393777
46.683885356602026
357.19811334039724
196.3704526954651
84.7474085360497
241.68628730914642
195.20817323427127
13.467832279407883
9.368836904286377
11.890498604132024
8.386828591601299
369.2430053007279
20.552128165350048
7.177850924966031
2.960539927281537
19.534889476365432
6.772234410399064
9.870963635664568
10.8540627579352
8.300858077588915
28.021735339015226
9.419733752928881
10.067965808189273
48.052951292721275
18.958878187788528
271.89707466505706
38.80446937968477
26.182777594871514
26.90298609968872
14.78

In [ ]:
cv2.destroyAllWindows()